In [21]:
import io
import os
from dotenv import load_dotenv
from google.cloud import storage
import pyarrow.parquet as pq


In [19]:
# Get the path to the credentials JSON key file from the environment variable
#key_path = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')

bucket_name = "pokemon_data_lake_subtle-sublime-383512"
file_path = "data/pokemon_info.snappy.parquet"

# Create a client object to access the bucket
client = storage.Client()

# Get the bucket object
bucket = client.get_bucket(bucket_name)

# Get the blob object representing the Parquet file
blob = bucket.blob(file_path)

# Download the Parquet file as a stream
stream = io.BytesIO()
blob.download_to_file(stream)
stream.seek(0)

# Convert the Parquet file to a Pandas dataframe
table = pq.read_table(stream, use_threads=True)
df = table.to_pandas()

In [17]:
df.head()

,id,name,height,weight,types
0,1,bulbasaur,7,69,grass
1,1,bulbasaur,7,69,poison
2,2,ivysaur,10,130,grass
3,2,ivysaur,10,130,poison
4,3,venusaur,20,1000,grass


In [23]:
path = 'data/pokemon_info.snappy.parquet'
file_name = os.path.basename(path)
pokemon_info = os.path.splitext(file_name)[0].split('.')[0]

print(pokemon_info)  # prints "pokemon_info"

pokemon_info


In [24]:
from prefect_gcp import GcpCredentials, GcsBucket